## Getting consensus sequences from alleles.loci files

**20170717**

I want to build a FASTA file that has the most common allele per locus across my RAD data, to use as a "reference genome" in ipyrad. I'm going to put together that code in this notebook, and ultimately put it into a script that I can reuse.

In [4]:
cd /mnt/hgfs/SHARED_FOLDER

/mnt/hgfs/SHARED_FOLDER


In [13]:
# get blocks of alleles separated by locus
test = open("test_all_loci.txt","r")
blocks = []
b = []
for line in test:
    if line.startswith('//'):
        if len(b) > 0:
            b.append(line)
            blocks.append(b)
        b = []
    else:
          b.append(line)
test.close()

In [14]:
# get index of starting base
firstline = blocks[0][0].strip()

# define function gets indeces of every time character appears in string
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

# get indeces of spaces in firstline
base_start = find(firstline, " ")[-1]+1

In [22]:
ex_hash = blocks[0][-1].strip()